In [ ]:
# faceMask hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 12/07/20

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data = hddm.utils.flip_errors(data)

# check dataset
data.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (angry) and facial expression (disgust)
ad_data = data.loc[(data['emotionRating'] == 'angry') & (data['expression'] == 'disgusted')]
ad_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
ad_data = ad_data[(ad_data.subj_idx != 'dqph4ch0mkofejw')]

In [ ]:
# define model
m_ad = hddm.HDDMRegressor(ad_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_ad.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_ad.print_stats()
m_ad.plot_posteriors()

In [ ]:
# save model stats
ad_stats = m_ad.gen_stats()
print(ad_stats)
ad_stats.to_csv('ad_5000/faceMask_hddm_faceRatings_maskFalsePositives_ad_5000.csv', index=True)

# save model posterior plots
m_ad.plot_posteriors(path='ad_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
ad_v_none, ad_v_lower, ad_v_upper = m_ad.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([ad_v_none, ad_v_lower, ad_v_upper])
plt.savefig('ad_5000/faceMask_hddm_faceRatings_maskFalsePositives_ad_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(ad_v_none < 0) = ", (ad_v_none.trace() < 0).mean())
print("P(ad_v_lower < 0) = ", (ad_v_lower.trace() < 0).mean())
print("P(ad_v_upper > 0) = ", (ad_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(ad_v_lower < ad_v_upper) = ", (ad_v_lower.trace() < ad_v_upper.trace()).mean())

In [ ]:
# subset dataset by emotion rating type (angry) and facial expression (happy)
ah_data = data.loc[(data['emotionRating'] == 'angry') & (data['expression'] == 'happy')]
ah_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
ah_data = ah_data[(ah_data.subj_idx != 'dqph4ch0mkofejw') & (ah_data.subj_idx != 'xs5439nm2v85thb')]

In [ ]:
# define model
m_ah = hddm.HDDMRegressor(ah_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_ah.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_ah.print_stats()
m_ah.plot_posteriors()

In [ ]:
# save model stats
ah_stats = m_ah.gen_stats()
print(ah_stats)
ah_stats.to_csv('ah_5000/faceMask_hddm_faceRatings_maskFalsePositives_ah_5000.csv', index=True)

# save model posterior plots
m_ah.plot_posteriors(path='ah_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
ah_v_none, ah_v_lower, ah_v_upper = m_ah.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([ah_v_none, ah_v_lower, ah_v_upper])
plt.savefig('ah_5000/faceMask_hddm_faceRatings_maskFalsePositives_ah_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(ah_v_none > 0) = ", (ah_v_none.trace() > 0).mean())
print("P(ah_v_lower < 0) = ", (ah_v_lower.trace() < 0).mean())
print("P(ah_v_upper > 0) = ", (ah_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(ah_v_lower < ah_v_upper) = ", (ah_v_lower.trace() < ah_v_upper.trace()).mean())

In [ ]:
# subset dataset by emotion rating type (angry) and facial expression (surprised)
ar_data = data.loc[(data['emotionRating'] == 'angry') & (data['expression'] == 'surprised')]
ar_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error) 
ar_data = ar_data[(ar_data.subj_idx != 'dqph4ch0mkofejw')]

In [ ]:
# define model
m_ar = hddm.HDDMRegressor(ar_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_ar.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_ar.print_stats()
m_ar.plot_posteriors()

In [ ]:
# save model stats
ar_stats = m_ar.gen_stats()
print(ar_stats)
ar_stats.to_csv('ar_5000/faceMask_hddm_faceRatings_maskFalsePositives_ar_5000.csv', index=True)

# save model posterior plots
m_ar.plot_posteriors(path='ar_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
ar_v_none, ar_v_lower, ar_v_upper = m_ar.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([ar_v_none, ar_v_lower, ar_v_upper])
plt.savefig('ar_5000/faceMask_hddm_faceRatings_maskFalsePositives_ar_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(ar_v_none < 0) = ", (ar_v_none.trace() < 0).mean())
print("P(ar_v_lower < 0) = ", (ar_v_lower.trace() < 0).mean())
print("P(ar_v_upper > 0) = ", (ar_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(ar_v_lower < ar_v_upper) = ", (ar_v_lower.trace() < ar_v_upper.trace()).mean())